In [ ]:
import xarray as xr
from osgeo import gdal
import pandas as pd

In [ ]:
#################################################################
#define output file format: 'netcdf' (one file per year, possible error if file too big ), 'geotiff' (daily files YYYYMMDD, might take large memory)
format = 'geotiff'         # <--------- change as needed ('netcdf, 'geotiff')

#define time range (inclusive)
year_start = 2001          # <--------- change as needed
year_end = 2022            # <--------- change as needed

#define if whole australia or cropped area (also used for files' names)
area = 'cropped'           # <--------- change as needed ('australia', 'cropped')

#define coordinates limits (values are ignored if 'australia' was chosen)
lat_min = -35.35           # <--------- change as needed
lat_max = -14.4            # <--------- change as needed

lon_min = 112.5            # <--------- change as needed
lon_max = 126.7            # <--------- change as needed

#path to folder where to save files
path = 'C:\\path\\folder'  # <--------- change as needed
#################################################################

In [ ]:
#fixed data for geotiff format
res = 0.005  # i.e., 500m spatial resolution 
wgs84_wkt = 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.01745329251994328,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]]'

In [ ]:
for y in range(year_start, year_end+1):
    print('Year: ', y)

    #open NetCDF file from THREDDS
    nc = xr.open_dataset('https://dapds00.nci.org.au/thredds/dodsC/ub8/au/FMC/mosaics/fmc_c6_{}.nc'.format(y))

    if area == 'cropped':
        #crop using coordinates (nearest coordinates)
        nc = nc.sel(latitude=slice(lat_max, lat_min), longitude=slice(lon_min, lon_max))


    #save locally as NetCDF (possible error if file is too big)
    if format == 'netcdf':
        nc.to_netcdf('{}\\lfmc_{}_{}.nc'.format(path, area, y))                           
    

    #save locally as single date geotiff files (might take a lot of space)
    elif format == 'geotiff':
        #extract geodata for creation of geotiff files
        nc_lat_min = round(min(nc.latitude.data), 3)                         
        nc_lat_max = round(max(nc.latitude.data), 3)                         
        nc_lon_min = round(min(nc.longitude.data), 3)                        
        nc_lon_max = round(max(nc.longitude.data), 3)                       

        geot = [nc_lon_min - res/2, res, 0., nc_lat_max + res/2, 0., -1*res] 
        
        #loop over dates in netcdf file
        for d in nc.time.data:

            d_str = pd.to_datetime(d).strftime('%Y%m%d')
            print(d_str)

            fmc_array = nc.sel(time=d).lfmc_median.data

            fmc_tif = gdal.GetDriverByName('GTiff').Create('{}\\lfmc_{}_{}.tif'.format(path, area, d_str), len(nc.longitude), len(nc.latitude), 1, gdal.GDT_Float32)               
            fmc_tif.SetGeoTransform(geot)  
            fmc_tif.SetProjection(wgs84_wkt)
            fmc_tif.GetRasterBand(1).WriteArray(fmc_array)
            fmc_tif.FlushCache() #save file
            del fmc_tif #close file


